# <div style="padding:20px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#AF601A;overflow:hidden;font-weight:500">TPS June 2022
</div>




 ### If you are a beginner, see my other notebook for imputation tutorial [notebook](https://www.kaggle.com/code/abdulravoofshaik/quick-eda-and-missing-values-tutorial). This work would not have been possible without the great insights from the following notebooks. Please consider upvoting the original work also
 ### The following cartoon depicts the overall framework for applying ensemble technique for imputation. I have used two neural network models (one with high Epoch, one with low Epoch), two LGBM models, one catboost model to train on the training dataset. Later I had applied Linear regression model for meta learning purpose. 
 
 ###  EDA is not the focus of this notebook. Please see this excellent [notebook](https://www.kaggle.com/code/masatomurakawamm/tps-jun22-application-of-masked-language-model) by Masato Murakawa if you are intersted in EDA.

 <div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana;">
    📌Important note: I have applied meta learning on the training dataset, as I wanted to find the best meta model which fits our dataset.
</div>
 


<img src="https://cdn-images-1.medium.com/max/946/1*TcGmOuMTARvshOY5bDN81Q.png">

source: https://towardsai.net/p/l/machine-learning-model-stacking-in-python

<div style="color:white;display:fill;border-radius:8px;
            background-color:#E59866;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>1.0 | Load data and Preprocessing</b></p>
</div>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import missingno as msno
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt 

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")
Target = pd.read_csv("../input/tabular-playground-series-jun-2022/sample_submission.csv", index_col='row-col')

### Below two csv files contains predict of the trained neural network on the whole dataset. For full details on the training can be found in this public [notebook](https://www.kaggle.com/code/alexryzhkov/lightautoml-nn-imputer-tps-june-22).


In [ ]:
train_pred_lowcnn= pd.read_csv("../input/metalearn-june2022/low_epoch_autoML_Pred_train_Test.csv") # low Epoch
train_pred_highcnn= pd.read_csv("../input/metalearn-june2022/highEpoch_autoML_Pred_train_Test.csv") # high Epoch

### Below, four csv files contain predictions of the trained model on the whole dataset. I have used XGB, Catboost and LGBM techniques for this purpose. You can get an intuition about the training procedure in my other public [notebook](https://www.kaggle.com/code/abdulravoofshaik/may-the-best-man-model-learn-ensemble4imputation).

In [ ]:
train_pred_xgb= pd.read_csv("../input/metalearn-june2022/Merged_train_Subsets_xgb.csv")
train_pred_cat= pd.read_csv("../input/metalearn-june2022/Merged_train_Subsets_catboost.csv")
train_pred_lowlgbm= pd.read_csv("../input/metalearn-june2022/Merged_train_Subsets_low_lgbm.csv")
train_pred_highlgbm= pd.read_csv("../input/metalearn-june2022/Merged_train_Subsets_20kLGBM.csv")

In [ ]:
# lets take a look at the data that we have imported
display(train_pred_xgb.head(), train_pred_cat.head(), train_pred_highlgbm.head(), train_pred_lowlgbm.head(),data.head())

In [ ]:
# Optmize memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

data = reduce_mem_usage(data)
Target = reduce_mem_usage(Target)
train_pred_xgb = reduce_mem_usage(train_pred_xgb)
train_pred_cat = reduce_mem_usage(train_pred_cat)
train_pred_highlgbm = reduce_mem_usage(train_pred_highlgbm)
train_pred_lowlgbm = reduce_mem_usage(train_pred_lowlgbm)
train_pred_lowcnn = reduce_mem_usage(train_pred_lowcnn)
train_pred_highcnn = reduce_mem_usage(train_pred_highcnn)

<div style="color:white;display:fill;border-radius:8px;
            background-color:#E59866;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>2.0 | Preprocessing</b></p>
</div>
Lets divide the data into four subsets.

In [ ]:
# this code snippet is taken from https://www.kaggle.com/code/martynovandrey/tps-jun-22-splitted-dataset-24x-faster. Consider upvoting the original author also
features = list(data.columns)
features_1, features_2, features_3, features_4 = [], [], [], []
F = [[], [], [], [], []]
for feature in features:
    for i in [1, 2, 3, 4]:
        if feature.split('_')[1] == str(i):
            F[i].append(feature)
df = [[], [], [], [], []]

for i in [1, 2, 3, 4]:
    df[i] = data[F[i]].copy()

In [ ]:
#removing f1,f2,f3 from the dataframes, as we will focus on columns starting with F_4_. 
F_4_list = [col for col in data.columns if col.startswith('F_4_')]
# adding xgb data
train_pred_xgb=train_pred_xgb[F_4_list]
train_pred_cat=train_pred_cat[F_4_list]
train_pred_highlgbm=train_pred_highlgbm[F_4_list]
train_pred_lowlgbm=train_pred_lowlgbm[F_4_list]
train_pred_highcnn=train_pred_highcnn[F_4_list]
train_pred_lowcnn=train_pred_lowcnn[F_4_list]


In [ ]:
# Lets add a suffix 
train_pred_xgb = train_pred_xgb.add_suffix('_xgb')
train_pred_cat = train_pred_cat.add_suffix('_cat')
train_pred_highlgbm = train_pred_highlgbm.add_suffix('_highlgbm')
train_pred_lowlgbm = train_pred_lowlgbm.add_suffix('_lowlgbm')
train_pred_highcnn = train_pred_highcnn.add_suffix('_highcnn')
train_pred_lowcnn = train_pred_lowcnn.add_suffix('_lowcnn')

In [ ]:
#merging all the F4 subsets into one dataframe
Merged_F4Subsets = pd.concat([train_pred_xgb, train_pred_cat, train_pred_highlgbm,train_pred_lowlgbm,train_pred_highcnn,train_pred_lowcnn, df[4]], axis=1)
Merged_F4Subsets.head()

In [ ]:
# pd.set_option('display.max_rows', 25)
print(Merged_F4Subsets.isnull().sum())

<div style="color:white;display:fill;border-radius:8px;
            background-color:#E59866;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.0 | Ensembling</b></p>
</div>

In [ ]:
dummy_df2=pd.DataFrame(0, index=np.arange(len(df[4])), columns=df[4].columns) # create zero filled dataframe
dummy_df2=df[4].copy()
for column in df[4].columns: 
    print('Processing Colunm Name : ', column)
    dummy_df=pd.DataFrame()
    
    if df[4][column].isnull().sum() == 0:
        print(df[4][column].isnull().sum())
        continue    # continue as no NaN values found in this column
    #collect all the columns start with selected column
    col_list = [col for col in Merged_F4Subsets.columns if col.startswith(column+str('_'))]
    print(col_list)
    dummy_df= pd.concat([Merged_F4Subsets[col_list],Merged_F4Subsets[column]],axis=1)
    col_nan_ix = dummy_df[dummy_df[column].isnull()].index  # identify the rows which has NaN in column F_1_0
    train_df = dummy_df.drop(col_nan_ix, axis = 0)  # this is the df which has no NaN
    test_df = dummy_df[dummy_df.index.isin(col_nan_ix)]
    print('R2 score for high CNN',r2_score(train_df[column+str('_highcnn')].values, train_df[column].values))
    print('R2 score for low CNN',r2_score(train_df[column+str('_lowcnn')].values, train_df[column].values))
    print('R2 score for highlgbm',r2_score(train_df[column+str('_highlgbm')].values, train_df[column].values))
    print('R2 score for lowlgbm',r2_score(train_df[column+str('_lowlgbm')].values, train_df[column].values))
    print('R2 score for xgb',r2_score(train_df[column+str('_xgb')].values, train_df[column].values))
    print('R2 score for cat',r2_score(train_df[column+str('_cat')].values, train_df[column].values))
    X= train_df[col_list].values
    y= train_df[column].values

    # Use KFold CV to avoid overfitting
    test_preds = None
    rmse_list = []
    fold_num = 1
    for train_index, val_index in KFold(n_splits=5, shuffle=True).split(X, y):
        X_train, y_train = X[train_index], y[train_index]
        X_val, y_val = X[val_index], y[val_index]     
        model = LinearRegression()
        model.fit(X_train,y_train)
        val_predictions = model.predict(X_val)
        r2_val = r2_score(y_val, val_predictions)
        print('Fold {fold_num} R2: {rmse:.4f}'.format(fold_num=fold_num, rmse=r2_val))
        rmse_list.append(r2_val)
        fold_num += 1
        # Use each fold's model to predict test values and add them to test_preds
        if test_preds is None:
            test_preds = model.predict(test_df.drop([column],axis=1))
        else:
            test_preds += model.predict(test_df.drop([column],axis=1))

    # Get average of predictions from KFold CV for submission
    test_preds /= fold_num
    dummy_df2[column][col_nan_ix] = model.predict(test_df.drop([column],axis=1))
    print('Average KFold rmse: {avg_rmse:.4f}'.format(avg_rmse = np.mean(np.array(rmse_list))))

### lets take a close the results. For example, for the column 'F_4_0'
### R2 score when CNN based model with high Epochs used is 0.9799020843221459
### R2 score when CNN based model with low Epochs used is  0.9362032002345319
### R2 score when LGBM based model with high number of n_estimators used is 0.8955968491804145
### R2 score when LGBM based model with low number of n_estimators used is 0.2907488927788481
### R2 score when XGB based model used is 0.5172610850741659
### R2 score when Catboost based model is 0.49500719294973083
### now the best part, the linear regressino model was able to give R2 score of 0.9899 for all the folds. We can see the same trend for all the other columns in this subset.

In [ ]:
#Now we use simple meanimputer for subset-1 and subset-3. Note that subset-2 has no missing values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(
        missing_values=np.nan,
        strategy='mean') 
for i in [1,3]:    
    df[i][:] = imp.fit_transform(df[i])

In [ ]:
Merged_Subsets = pd.concat([df[1], df[2], df[3], dummy_df2], axis=1)
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = Merged_Subsets.loc[row, col]

submission.to_csv('meta_lr_submission2.csv')

<div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana;">
    📌 Now your turn. I have used Linear regression for meta learning, but you can use some other techniques such as XGB, Catboost. Make sure you do not overfit.
</div>

### The Lb score is 0.87 which is lower. This could be two things:
1) The model is overfitted <br>
2) The public LB score is misleading. <br>

I belive the number two is true. Because, we have applied Kfold and all the folds are shown very good fit with training data. We have seen similar LB in previous competitions. 
Way forward, fine tune Neural networks model


## Work in progress
<img src="https://pbs.twimg.com/media/FPBbO_zWQAokwwc?format=jpg&name=small">


<div style="color:white;display:fill;border-radius:8px;
            background-color:#E59866;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.0 | References</b></p>
</div>

https://towardsdatascience.com/using-the-missingno-python-library-to-identify-and-visualise-missing-data-prior-to-machine-learning-34c8c5b5f009 <br>
https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4 <br>
https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python/notebook <br>
https://www.kaggle.com/code/residentmario/using-missingno-to-diagnose-data-sparsity/notebook <br>
https://www.analyticsvidhya.com/blog/2021/05/dealing-with-missing-values-in-python-a-complete-guide/ <br>
https://www.kaggle.com/code/calebreigada/getting-started-eda-preprocessing <br>
https://medium.com/swlh/impute-missing-values-the-right-way-c63735fccccd <br>

